In [1]:
import arcpy
import pandas as pd

In [2]:
places = r'D:\projects\foursquare-geoenrichment-loading\data\processed\geoenrichment\foursquare.gdb\places'
xlsx_pth = r'D:\projects\foursquare-geoenrichment-loading\data\external\XXX_FOURSQUARE_2023_Metadata.xlsx'

In [23]:
xlsx_sheet = "Layer"
col_name = 'SOURCE_FIELD'
col_length = 'FIELD_LENGTH'

# read the excel workbook sheet into a dataframe
df = pd.read_excel(xlsx_pth, engine='openpyxl', sheet_name=xlsx_sheet, header=0)

# get a data frame of field names and lengths from the feature class
df_len = pd.DataFrame([(f.name, f.length) for f in arcpy.ListFields(places)], columns=['field_name', 'field_length']).set_index('field_name', drop=True)

# create a data frame of lengths in the excel file and intropsected lengths
df_update = df[[col_name, col_length]].join(df_len, on=col_name, how='left')

# by comparing listed to observed, get a data frame of changed fields
df_changes = df_update[df_update[col_length] != df_update['field_length']]

# update the fields with an update value in original column...ensuring do not create nulls
fltr = df_update['field_length'].notnull()
df_update.loc[fltr, col_length] = df_update.loc[fltr, 'field_length']

# fill in the introspected values in the original column
df[col_length] = df_update[col_length]

# replace the lengths with introspected values
df[col_length] = df[col_name].apply(lambda fld_nm: fc_dict.get(fld_nm))

# write the updated data frame back to the workbook sheet
with pd.ExcelWriter(xlsx_pth, mode='a', engine='openpyxl', if_sheet_exists='replace') as xlsx_write:
    df.to_excel(xlsx_write, sheet_name=xlsx_sheet, index=False)